In [1]:
import os, io, json, urllib, numpy as np, codecs, os.path,time
def h_(cited):
    h=0
    cited.sort(reverse=True)
    for i in range(1,len(cited)+1):
        if cited[i-1] >= i:
            h=i
    return h

In [2]:
def get_citation(issn,out):
    global item, all_items, url, url_next
    item={}; all_items=[];
    url='http://api.crossref.org/journals/'+issn+'/works?rows=1000&cursor=*'
    response = urllib.urlopen(url).read()
    if response=='Resource not found.':
        return
    json_result=json.loads(response)
    publications_total = json_result['message']['total-results']
    #from_year='2012';until_year='2016'
    #url='http://api.crossref.org/journals/'+issn+'/works?filter=from-pub-date:'+from_year+'-01-01,until-pub-date:'+until_year+'-12-31&rows=1000&cursor=*'    
    #response = urllib.urlopen(url).read()    
    #json_result=json.loads(response)
    #publications_year = json_result['message']['total-results']
    all_items = json_result['message']['items']
    if publications_total>1000:
        while True:
            next_cursor=json_result['message']['next-cursor']
            url_next=url[:-1]+next_cursor
            response = urllib.urlopen(url_next).read()
            json_result=json.loads(response)
            if 'items' not in json_result['message']: return
            items = json_result['message']['items']
            all_items = all_items + items
            if len(items)==0:
                break
    if len(all_items)>0:
        f=codecs.open(out+'/'+issn+'.txt','a','utf-8')
        for item in all_items:
            year_print=9999;year_online=9999;
            if 'published-print' in item:
                year_print=item['published-print']['date-parts'][0][0]
            if 'published-online' in item:
                year_online=item['published-online']['date-parts'][0][0]
            year=str(min(year_print,year_online))
            doi=item['DOI']
            issn=','.join(item['ISSN'])
            journal=','.join(item['container-title'])
            pub=item['publisher']; pub=''if pub==None else pub
            typ=item['type']
            cite=str(item['is-referenced-by-count'])
            res= year+'\t'+doi+'\t'+issn+'\t'+journal+'\t'+pub+'\t'+typ+'\t'+cite+'\n'
            f.write(res)    
        f.close()

In [33]:
with open('queryResults.csv') as f:
    lines = f.readlines()
lines.remove(lines[0])
len(lines)

68758

In [ ]:
out='journals'
i=0
item={}
all_items=[]
for line in lines:
    issn=line[2:11]
    if i % 10 == 0 : print i,
    fname=out+'/'+issn+'.txt'
    while True:
        try:
            get_citation(issn,out)
        except:
            time.sleep(100)
            continue
        break
    lines.remove(line)
    i=i+1

0 10

In [30]:
url

'http://api.crossref.org/journals/1755-7682/works?rows=1000&cursor=*'

In [31]:
url_next

u'http://api.crossref.org/journals/1755-7682/works?rows=1000&cursor=AoE+aHR0cDovL2R4LmRvaS5vcmcvMTAuMzgyMy8yMzI4'